In [1]:
from z3 import *
import random
import math 
import numpy as np

SLOW = 1
FAST = 10
SAFE_DIST = 100
MAX_X_Y = 500
MIN_X_Y = -500
TEMPO = 1 # sec
Mode, (M_SLOW, M_FAST) = EnumSort("Mode", ("SLOW", "FAST"))

In [2]:
def ranAng():
    ang = [i*15 for i in range(int(360/15))]
    np.random.shuffle(ang)
    c=ang[0]
    
    return c
ranAng()

270

In [3]:
def ranPos():
    return random.uniform(MIN_X_Y, MAX_X_Y)

ranPos()

-425.79621287886704

In [4]:
def posAng(d):
    return (d % 360)

posAng(-10)

350

In [5]:
def declare(i):
    boats = 3 
    trace = {}
    for b in range(boats):
        trace[b] = {}
        trace[b]["v"] = Const('velocidade '+str(i),Mode)
        trace[b]["a"] = Int('angulo '+str(i))
        trace[b]["t"] = Real('time '+str(i))
        trace[b]["px"] = Real('posx '+str(i))
        trace[b]["py"] = Real('posy '+str(i))
        trace[b]["ds"] = Real('distSeg '+str(i))
        
    return trace

declare(0)

{0: {'v': velocidade 0,
  'a': angulo 0,
  't': time 0,
  'px': posx 0,
  'py': posy 0,
  'ds': distSeg 0},
 1: {'v': velocidade 0,
  'a': angulo 0,
  't': time 0,
  'px': posx 0,
  'py': posy 0,
  'ds': distSeg 0},
 2: {'v': velocidade 0,
  'a': angulo 0,
  't': time 0,
  'px': posx 0,
  'py': posy 0,
  'ds': distSeg 0}}

In [6]:
def init(s):
    r = []
    for i in range(len(s)):
        angle=ranAng()
        r.append(s[i]["v"] == M_FAST)
        r.append(s[i]["a"] == angle)
        r.append(s[i]["px"] == ranPos())
        r.append(s[i]["py"] == ranPos())
        r.append(s[i]["t"] == 0)
        r.append(s[i]["ds"] == SAFE_DIST)

    return And(r)

init(declare(0))

And(velocidade 0 == FAST,
    angulo 0 == 120,
    posx 0 == 705736954671627/3125000000000,
    posy 0 == 2593742342190751/100000000000000,
    time 0 == 0,
    distSeg 0 == 100,
    velocidade 0 == FAST,
    angulo 0 == 195,
    posx 0 == 7338306960934881/50000000000000,
    posy 0 == -14750214961379527/500000000000000,
    time 0 == 0,
    distSeg 0 == 100,
    velocidade 0 == FAST,
    angulo 0 == 30,
    posx 0 == -1655508876502893/5000000000000,
    posy 0 == 3432534829291533/10000000000000,
    time 0 == 0,
    distSeg 0 == 100)

In [7]:
def trans(ant, curr):
    angs = [i*15 for i in range(int(360/15))]
    
    unt=[]
    t = []
    estado = []
    
    #timed
    for i in range(len(ant)):
        t.append(curr[i]["v"] == ant[i]["v"])
        t.append(curr[i]["ds"] == ant[i]["ds"])
        t.append(curr[i]["t"] - ant[i]["t"] == TEMPO) #1 sec
    
        cons = []
        consf=[]
        
        ''''''
        
        for c in range(len(angs)):
            con1 = []
            ag=angs[c]
            con1.append(ant[i]["a"] == ag)
            con1.append(curr[i]["a"] == ag)
            #slow
            sx = 1 * math.cos(math.radians(ag))
            sy = 1 * math.sin(math.radians(ag))
            #fast
            fx = 10 * math.cos(math.radians(ag))
            fy = 10 * math.sin(math.radians(ag)) 
            
            con1.append(curr[i]["px"] == ant[i]["px"] + sx)
            con1.append(curr[i]["py"] == ant[i]["py"] + sy)
            con1.append(curr[i]["px"] == ant[i]["px"] + fx)
            con1.append(curr[i]["py"] == ant[i]["py"] + fy)
            
            cons.append(And(con1))
        
        
        t.append(Or(cons))
        
    t = And(t)

    #UNTIMED
    
    #ant != curr
    for i in range(len(ant)):
        estado.append(ant[i]["px"] != curr[i]["px"])
        estado.append(ant[i]["py"] != curr[i]["py"])
        estado.append(ant[i]["v"] != curr[i]["v"])
        estado.append(ant[i]["a"] != curr[i]["a"])
    
    for i in range(len(ant)):
        #possibilidades 1º fast->fast ff
        #2º slow->slow ss 
        #3º slow->fast sf
        #4º fast-slow fs
        ''' '''
        unt.append(curr[i]["px"] == ant[i]["px"])
        unt.append(curr[i]["py"] == ant[i]["py"])
        unt.append(curr[i]["ds"] == ant[i]["ds"])
        unt.append(curr[i]["t"] == ant[i]["t"])
        
        #fast->fast
        ff = []
        ff.append(And(ant[i]["v"] == M_FAST, curr[i]["v"] == M_FAST))
        ff.append(curr[i]["a"] == ant[i]["a"])
        ff = And(ff)
        
        #slow -> slow
        ss = []
        ss.append(And(ant[i]["v"] == M_SLOW, curr[i]["v"] == M_SLOW))
        ss.append(curr[i]["a"] == ant[i]["a"])
        ss = And(ss)

        #slow -> fast
        sf = []
        sf.append(And(ant[i]["v"] == M_SLOW, curr[i]["v"] == M_FAST))
        sf.append(curr[i]["a"] == ant[i]["a"])
        sf = And(sf)

        #fast -> slow
        fs = []
        
        fs.append(And(ant[i]["v"] == M_FAST, curr[i]["v"] == M_SLOW))
        fs.append(Or(curr[i]["a"] == posAng(ant[i]["a"]+15), curr[i]["a"] == posAng(ant[i]["a"]-15)))
        fs = And(fs)
        
        unt.append(Or(ff, ss, sf, fs))
        
    unt = And(And(unt), And(estado))
      
    result = (Or(unt, t))
    result=t
    
    return result

In [8]:

def trace(declare, init, trans, k):
    solver = Solver()
    trace = [declare(i) for i in range(k)]
    solver.add(init(trace[0]))
    
    for i in range(k-1):
        solver.add(trans(trace[i], trace[i+1]))
        
    if solver.check() == sat:
        m = solver.model()
        
        for i in range(k):
            for b in range(len(trace[i])):
                for v in trace[i][b]:
                    if trace[i][b][v].sort() != RealSort():
                        print('Boat_'+b+'_'+v,' = ', m[trace[i][b][v]])
                    else: 
                        print('Boat_'+b+'_'+v,' = ', float(m[trace[i][b][v]].numerator_as_long())/float(m[trace[i][b][v]].denominator_as_long()))
    else: 
        print(unsat)

trace(declare, init, trans, 10)

unsat
